In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Transfer into xlsx format for convenience
pd.read_csv("./data_original.csv").to_excel("data_original.xlsx",index=False)

In [9]:
df = pd.read_excel("data_original.xlsx")

In [10]:
df.sample(3)

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
1287,10.1039/C4CP01679C,7/2/2014,Physical Chemistry Chemical Physics,"The solvatochromic, spectral, and geometrical ...",DMF,0.0,NaN,rsc_cde_tables,CN(C)C=O,1.431,n
2902,10.1016/j.jlumin.2018.04.026,4/12/2018,Journal of Luminescence,EU2O3DOPEDBRIGHTORANGEREDLUMINESCENTLITHIUMALU...,Oxygen,0.0,NaN,el_cde_tables,O,1.3634,n
1992,10.1016/j.jct.2004.09.021,1/5/2005,The Journal of Chemical Thermodynamics,PHYSICALPROPERTIESANISOLENALKANESTEMPERATURESB...,Octane,0.0,NaN,el_cde_tables,NaN,1.3947,nD


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DOI                     5000 non-null   object 
 1   Date                    4593 non-null   object 
 2   Journal                 4593 non-null   object 
 3   Title                   4593 non-null   object 
 4   Name                    4996 non-null   object 
 5   measurement_error       5000 non-null   float64
 6   measurement_wavelength  588 non-null    object 
 7   measurement_method      5000 non-null   object 
 8   normalised_name         2946 non-null   object 
 9   raw_value               5000 non-null   object 
 10  specifier               5000 non-null   object 
dtypes: float64(1), object(10)
memory usage: 429.8+ KB


In [12]:
# Для удобста ватаскивания значений через iloc
column_index_dic = {col_name:index for index,col_name in enumerate(df.columns)}

In [13]:
# В столбце "Name" есть пропущенные значения - 4 штуки 
print("Missed values ['Name']: ", df["Name"].isna().sum())
# В столбце "Name", заполним их исходя их столбца "normalized name"

import requests 

def convert_smiles_to_name(smi):
    try:
        url ="https://cactus.nci.nih.gov/chemical/structure/" + smi+"/iupac_name" 
        res = requests.get(url)
        return res.text.strip()
    except:
        return 'name not found'
    
for index in df[df["Name"].isna()].index:
    df.iloc[index,4] = convert_smiles_to_name(df.iloc[index,column_index_dic["normalised_name"]])


Missed values ['Name']:  4


In [14]:
# Теперь заполним пропуски в "normalised_name" через "name"

# В столбце "Name" есть пропущенные значения - 4 штуки 
print("Missed values ['normalised_name']: ", df["normalised_name"].isna().sum())
# В столбце "Name", заполним их исходя их столбца "normalized name"

def convert_names_to_smiles(norm_name):
    try:
        url ="https://cactus.nci.nih.gov/chemical/structure/" + norm_name+"/smiles" 
        res = requests.get(url)
        return res.text.strip()
    except:
        return 'name not found'
 
for index in df[df["normalised_name"].isna()].index:
    df.iloc[index,column_index_dic["normalised_name"]] = convert_names_to_smiles(df.iloc[index,column_index_dic["Name"]])


Missed values ['normalised_name']:  2054


In [15]:
# на всякий случай, если есть, избавляемся от пробелов по краям строк
df["DOI"] = df["DOI"].apply(lambda x: x.strip())

## Обработка DOIs - первая интераця 

In [20]:
# Обраюотка DOI
# Если есть пробелы, то начинаем обработку 
from bs4 import BeautifulSoup
# индексы строк с пробелами в DOI
index_rows= df[df["DOI"].apply(lambda x: " " in x)].index

slice_wrong_DOIs = df.iloc[index_rows,column_index_dic["DOI"]]
correct_slice_DOIs = []
for doi in slice_wrong_DOIs.values:
    try:
        correct_doi = []
        for symbol in doi:
            correct_doi.append(symbol)
            # все DOI длинее 11 символов)
            if len(correct_doi)>11:
                response = requests.get(f"https://sci-hub.ru/{''.join(correct_doi)}")
                tag_value = BeautifulSoup(response.content, 'html.parser').find('title').get_text()
                if tag_value != "Sci-Hub: статья не найдена":
                    correct_slice_DOIs.append("".join(correct_doi))
                    break
    except:
        print("nothing")

# Вставляем исправленные значения в DOI
df.iloc[index_rows,column_index_dic["DOI"]] = correct_slice_DOIs

In [21]:
# Проверяем, какие статьи не ищутся и вытаскиваем индекс строк, таких статей
responces = []
doi_problems = []
for doi in df["DOI"]:
    response = requests.get(f"https://sci-hub.ru/{doi}")
    tag_value = BeautifulSoup(response.content, 'html.parser').find('title').get_text()
    if tag_value == "Sci-Hub: статья не найдена":
        doi_problems.append(doi)
    responces.append(tag_value)

In [22]:
# Проблемнеы DOIs, есть дубликаты - удаляем
doi_problems = set(doi_problems)

In [24]:
# Собственно проблемные DOIs

# План такой: убрать слова после цифр, эти DOIs прогнать через sci-hub
# обновлять journal, title и date через crossref

processed_doi_problems = dict()
for doi in doi_problems:
    uppers = [l for l in doi[::-1] if l.isupper()]
    if len(uppers)>0: 
        processed_doi_problems[doi] = doi[:doi.find(uppers[0])]
    else:
        processed_doi_problems[doi] = doi

In [26]:
# Проверяем обработанные статьи ячейкой выше processed_doi_problems

problematic_doi = []
for doi in list(processed_doi_problems.values()):
    response = requests.get(f"https://api.crossref.org/works/{doi}")
    if response.ok:
        data = response.json()["message"]
        print(f"Title: {data['title'][0]}")
        print(f"Publication Date: {data['created']['date-parts'][0][0]}")
        print(f"Journal: {data['container-title'][0]}")
        print(f"DOI: {data['DOI']}")
    else:
        problematic_doi.append(doi)
        print("DOI not found.")

Title: Heat treatment effect on the structural and optical properties of AgInSe2 thin films
Publication Date: 2002
Journal: Vacuum
DOI: 10.1016/s0042-207x(01)00417-1
Title: The light transmission and distribution in an optical fiber coated with TiO2 particles
Publication Date: 2003
Journal: Chemosphere
DOI: 10.1016/s0045-6535(02)00641-0
DOI not found.
Title: Theoretical study of structural, electronic and optical properties of InxGa1-xN alloys
Publication Date: 2018
Journal: Optik
DOI: 10.1016/j.ijleo.2018.08.083
Title: The research on syntheses and properties of novel epoxy/polymercaptan curing optical resins with high refractive indices
Publication Date: 2002
Journal: Polymer
DOI: 10.1016/s0032-3861(01)00573-0
Title: Realization of tunable optical channel drop filter based on photonic crystal octagonal shaped structure
Publication Date: 2018
Journal: Optik
DOI: 10.1016/j.ijleo.2018.06.146
Title: High-performance bimetallic film surface plasmon resonance sensor based on film thickness

In [27]:
# Проблемные статьи))
print(problematic_doi)

['10.3389/fpls.2014.00', '10.1039/C6NR08470', '10.1039/C6TC05201', '10.1038/ncomms8']


In [28]:
# Заменяем старые DOIs новыми, 
# но остается два неразрешенных DOIs - ['10.1038/srep28', '10.3389/fpls.2014.00']
for key,value in processed_doi_problems.items():
    df.loc[df["DOI"]==key,"DOI"] = value

In [29]:
# ['10.3389/fpls.2014.00', '10.1039/C6NR08470', '10.1039/C6TC05201', '10.1038/ncomms8'] удаляем эти DOIs (строки)
for doi_prob in problematic_doi:
    df = df.drop(df[df["DOI"]==doi_prob].index)

In [45]:
# Заполянем пропуски в (Date, Journal, Title)
# Индексы пропусков у них одинаковые

# Индексы пропусков у них одинаковые
missed_index = df[df["Journal"].isna()].index

In [47]:
# Создаем список, который потом заполнить пропуски
problematic_doi_2 = []
values_fill = []
for index in missed_index:
    response = requests.get(f"https://api.crossref.org/works/{df.loc[index,'DOI']}")
    if response.ok:
        data = response.json()["message"]
        values_fill.append([data['created']['date-parts'][0][0],
                            data['container-title'][0],data['title'][0]])
    else:
        problematic_doi_2.append(df.loc[index,'DOI'])
        print("DOI not found.")

DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.


In [49]:
# Удаляем эти DOIs (строки)
for doi_prob in problematic_doi_2:
    df = df.drop(df[df["DOI"]==doi_prob].index)

In [51]:
df.to_excel("data_mod1.xlsx")

In [3]:
df = pd.read_excel("data_mod1.xlsx")

In [4]:
# Индексы пропусков у них одинаковые - но изменились после удаления, поэтому заново
missed_index = df[df["Journal"].isna()].index

In [10]:
# Создаем список, который потом заполнить пропуски (повторно, так как удалили некоторые строки)
import time
import requests

problematic_doi_2 = []
values_fill = []
for index in missed_index:
    response = requests.get(f"https://api.crossref.org/works/{df.loc[index,'DOI']}")
    if response.ok:
        data = response.json()["message"]
        values_fill.append([data['created']['date-parts'][0][0],
                            data['container-title'][0],data['title'][0]])
    else:
        problematic_doi_2.append(df.loc[index,'DOI'])
        print("DOI not found.")

In [12]:
# Заполянем пропуски в (Date, Journal, Title)
df.loc[missed_index,"Date"] = [x[0] for x in values_fill]
df.loc[missed_index,"Journal"] = [x[1] for x in values_fill]
df.loc[missed_index,"Title"] = [x[2] for x in values_fill]

In [15]:
df.to_excel("data_mod1.xlsx")

## Заполняем пропуски в SMILES (через pubchempy)

In [16]:
import pubchempy as pcp

def fill_smiles(mol_names):
    smile_list = []
    for mol_name in mol_names.values:
        try:
            smile = pcp.get_properties('CanonicalSMILES', mol_name, 'formula')[0]['CanonicalSMILES']
            smile_list.append(smile)
        except:
            smile_list.append(None)
    return smile_list

mol_names = df.loc[df["normalised_name"]=="<h1>Page not found (404)</h1>","Name"]
smiles = fill_smiles(mol_names)
df.loc[df["normalised_name"]=="<h1>Page not found (404)</h1>","normalised_name"] = smiles

In [18]:
df.to_excel("data_mod1.xlsx")

## Заполянем пропуски в measurement_length

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4985 entries, 0 to 4984
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DOI                     4985 non-null   object 
 1   Date                    4985 non-null   object 
 2   Journal                 4985 non-null   object 
 3   Title                   4985 non-null   object 
 4   Name                    4985 non-null   object 
 5   measurement_error       4985 non-null   float64
 6   measurement_wavelength  588 non-null    object 
 7   measurement_method      4985 non-null   object 
 8   normalised_name         4680 non-null   object 
 9   raw_value               4985 non-null   object 
 10  specifier               4985 non-null   object 
dtypes: float64(1), object(10)
memory usage: 428.5+ KB


In [20]:
import re

def get_wavelength(df_row):
  
  """Заполняем пропуски в колонке measurement_wavelength"""
    
  if pd.notna(df_row['measurement_wavelength']):
    num = re.findall(r'[+]?\d*\.?\d+|\d+', df_row['measurement_wavelength'])
    if 'μm'in df_row['measurement_wavelength']:
      return float(num[0])*1000
    else: 
      return float(num[0])
  else:
    if re.search(r'\s*n*[D]', df_row['specifier']):
      return 589
    elif re.search(r'[\s(]n.*d', df_row['specifier']):
      return 588
    elif re.search(r'[\s(]n.*[fF]', df_row['specifier']):
      return 486
    elif re.search(r'[\s(]n.*g', df_row['specifier']):
      return 436
    elif re.search(r'[\s(]n.*e', df_row['specifier']):
      return 546
    num = re.findall(r'[+]?\d*\.?\d+|\d+', df_row['specifier'])
    if len(num)!=0:
      if 'nm' in df_row['specifier']:
        return float(num[0])
      elif 'μm' in df_row['specifier'] or 'μ  m'in df_row['specifier']:
        return float(num[0])*1000
      elif ('Å' in df_row['specifier']) or ('Å' in df_row['specifier']):
        return float(num[0])/10


In [21]:
#создаем новую колонку, куда копируем имеющиеся значения 'measurement_wavelength' и дозаполняем пропуски
df['measurement_wavelength'] = df.apply(lambda row: get_wavelength(row), axis=1)

## Заполняем пропуски в колонку 'measurement_error'

In [24]:
df["measurement_error"] = df["measurement_error"].fillna(0)

In [25]:
def get_measurement_error(row):
    values = re.findall(r"[+]?\d*\.?\d+|\d+", row['raw_value'])
    values = [float(x) for x in values]
    if len(values) == 1:
        return 0
    # если погрешность
    elif values[0] > values[1]:
        if row['measurement_error']==0:
            return values[1]
        else:
            return row['measurement_error']
    # если интервал
    elif values[0] < values[1]:
        if row['measurement_error']==0:
            return (values[1] - values[0])/2
        else:
            return row['measurement_error']
    else:
        if row['measurement_error']!=0:
            return row['measurement_error']
        else:
            return 0

In [26]:
# Новые заненяи у measurement_error
df["measurement_error"] = df.apply(lambda row: get_measurement_error(row), axis=1)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4985 entries, 0 to 4984
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DOI                     4985 non-null   object 
 1   Date                    4985 non-null   object 
 2   Journal                 4985 non-null   object 
 3   Title                   4985 non-null   object 
 4   Name                    4985 non-null   object 
 5   measurement_error       4985 non-null   float64
 6   measurement_wavelength  1273 non-null   float64
 7   measurement_method      4985 non-null   object 
 8   normalised_name         4680 non-null   object 
 9   raw_value               4985 non-null   object 
 10  specifier               4985 non-null   object 
dtypes: float64(2), object(9)
memory usage: 428.5+ KB


In [28]:
df.to_excel("data_mod1.xlsx")

In [ ]:
# Удаляем дубликаты строк по DOI и Name и measurement_method и normalised_name и specifier
df = df.drop_duplicates(subset=["DOI","Name"])

**Getting descriptors from APIs**

Since one of the tasks was to collect 1000 descriptors, in addition to the usual APIs with a small number of parameters for each molecule (ChemSpider, PubChem, RDKit, Materials Project) it was necessary to choose at least one big API with calculated molecular descriptors. Examples of such APIs are PaDEL-descriptor [1] and molecular descriptor calculator Mordred [2]. PaDEL-descriptor is currently considered outdated and Mordred is regarded as a faster and more convenient tool, so it was decided to settle on it. Among classical APIs Materials Project was chosen due to the availability of parameters for a large number of inorganic molecules. RDKit was also chosen, as it is appealing because it has a large number of descriptors and is open-source, while ChemSpider contains few descriptors, and PubChem is also slow.

[1] - https://onlinelibrary.wiley.com/doi/10.1002/jcc.21707

[2] - https://jcheminf.biomedcentral.com/articles/10.1186/s13321-018-0258-y

In [1]:
# Import APIs
from mp_api.client import MPRester
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import Chem
from mordred import Calculator, descriptors

# Import other things 
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# Firstly we have to load cleansed data
df = pd.read_excel('data_mod1.xlsx')

In [3]:
df

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/j.foodhyd.2014.07.012,7/24/2014,Food Hydrocolloids,PHYSICOCHEMICALCHARACTERIZATIONANTIMICROBIALAC...,p-cimene,0.0,633.0,el_mylogic,CC(C)c1ccc(C)cc1,1.471,Refractive index
1,10.1016/j.jct.2015.04.010,4/27/2015,The Journal of Chemical Thermodynamics,LIQUIDLIQUIDPHASEEQUILIBRIUMCRITICALBEHAVIORBI...,D2O,0.0,632.8,el_mylogic,O([2H])[2H],1.4173,n
2,10.1016/j.apcatb.2014.06.020,6/20/2014,Applied Catalysis B: Environmental,ENHANCEDPHOTOCATALYTICPERFORMANCETIO2BASEDSYNE...,TiO2,0.0,396.0,el_mylogic,"[['O', 2.0], ['Ti', 1.0]]",1.6,Refractive index
3,10.1016/S0022-2860(03)00155-8,4/2/2003,Journal of Molecular Structure,SOLVENTEFFECTSPHOTOCHEMICALFLUORESCENCEPROPERT...,Pyridine,0.0,14.0,el_mylogic,c1ccncc1,1.509,"Refractive index, n D"
4,10.1016/j.powtec.2006.10.021,10/24/2006,Powder Technology,PARTICLESIZEDISTRIBUTIONBYSPACETIMEDEPENDENTEX...,SiO2,0.0,NaN,el_cde_tables,"[['O', 2.0], ['Si', 1.0]]",1.44,Refractive Index
...,...,...,...,...,...,...,...,...,...,...,...
4980,10.1016/j.mseb.2018.10.016,11/6/2018,Materials Science and Engineering: B,HIGHLYTRANSPARENTZINCNITRIDETHINFILMSBYRFMAGNE...,Zn3N2,0.0,500.0,el_mylogic,[N-]=[Zn].[N-]=[Zn].[Zn+2],1.32,Refractive index (n) (500 nm)
4981,10.1016/j.opelre.2017.06.002,2017,Opto-Electronics Review,Performance study of a liquid-core Bragg fiber...,sucrose,0.0,NaN,snowball,OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C...,1.4418,refractive index
4982,10.1016/j.matchemphys.2017.03.036,3/26/2017,Materials Chemistry and Physics,MODIFICATIONSINOPTICALSTRUCTURALPROPERTIESPMMA...,PMMA,0.0,NaN,el_cde_tables,COC(=O)[C](C)C,1.43,Refractive index
4983,10.1016/j.enconman.2010.06.016,6/25/2010,Energy Conversion and Management,CATALYTICPYROLYSISLDPELEADSVALUABLERESOURCEREC...,ZnO,0.0,NaN,el_mylogic,O|[Zn++]|O,1.453,Refractive index (η)


Descriptors from Materials Project

In [4]:
MP_API_KEY = "tOIX0eqKRxvebjdAnUPan5jZRJ6MwAAz"

In [6]:
with MPRester(MP_API_KEY) as mpr:
    all_fiels = mpr.summary.available_fields

Firstly, let's check what we can get

In [7]:
print(all_fiels)

['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_total

It is obvious that we don't need most of these field, such as several formulas, different kinds of metadata, elements and so on, so let's get rid of them

In [8]:
useless_fiels = [
    "builder_meta",
    "elements",
    "nelements",
    "composition",
    "composition_reduced",
    "formula_pretty",
    "formula_anonymous",
    "chemsys",
    "symmetry",
    "property_name",
    "material_id",
    "deprecated",
    "deprecation_reasons",
    "last_updated",
    "origins",
    "warnings",
    "task_ids",
    "database_IDs",
    "decomposes_to",
    "es_source_calc_id",
    "has_reconstructed",
    "possible_species",
    "has_props",
]


In [9]:
# These are remaining useful descriptors
useful_fields = list(set(all_fiels) - set(useless_fiels))


In [10]:
# We will store normalised names of the columns and MPData docs for future dataframe
docs = []
names = []


Now we can download all data from Materials Project, storing names and docs

In [ ]:
for name in tqdm(df.normalised_name):
    try:
        doc = mpr.summary.search(formula=name, fields=useful_fields)
        if doc != []:
            names += [name]
            docs += [doc]
    except:
        pass

Now we can store our data in a dataframe with names and all of the descriptors

In [12]:
df_mat_proj = pd.DataFrame(columns=["normalised_name"] + useful_fields)


In [13]:
for i, doc in tqdm(enumerate(docs)):
    data = []
    for field in useful_fields:
        data.append(getattr(doc[0], field))
    df_mat_proj.loc[len(df_mat_proj.index)] = [names[i]] + data


482it [00:03, 130.09it/s]


In [14]:
# Dataframe with all descriptors from Materials Project
df_mat_proj


,normalised_name,xas,structure,grain_boundaries,theoretical,is_magnetic,weighted_surface_energy,vbm,num_magnetic_sites,weighted_work_function,...,energy_per_atom,energy_above_hull,dos_energy_up,total_magnetization_normalized_formula_units,weighted_surface_energy_EV_PER_ANG2,types_of_magnetic_species,shape_factor,e_total,bandstructure,g_vrh
0,ClC(Cl)(Cl)Cl,[edge=<Edge.K: 'K'> absorbing_element=Element ...,"[[23.92950295 -9.16403282 11.10570111] C, [18....",None,False,False,NaN,-1.9445,0,NaN,...,-3.966848,0.000000,None,0.000000,NaN,[],NaN,NaN,setyawan_curtarolo=BandStructureSummaryData(ta...,NaN
1,CCCCCCCCCCCC,[edge=<Edge.K: 'K'> absorbing_element=Element ...,"[[0. 0. 1.95076825] C, [0. ...",None,False,False,0.019117,None,0,5.025481,...,-9.220297,0.008368,None,0.000000,0.001193,[],72.510497,NaN,setyawan_curtarolo=BandStructureSummaryData(ta...,95.525895
2,ClC(Cl)Cl,None,"[[5.09346712 7.62057649 7.01241824] C, [1.3954...",None,True,False,None,-1.0493,0,None,...,-4.276994,0.018717,None,0.000000,None,[],None,NaN,None,None
3,CCCCCCC,[edge=<Edge.K: 'K'> absorbing_element=Element ...,"[[0. 0. 1.95076825] C, [0. ...",None,False,False,0.019117,None,0,5.025481,...,-9.220297,0.008368,None,0.000000,0.001193,[],72.510497,NaN,setyawan_curtarolo=BandStructureSummaryData(ta...,95.525895
4,C1CCCC1,[edge=<Edge.K: 'K'> absorbing_element=Element ...,"[[0. 0. 1.95076825] C, [0. ...",None,False,False,0.019117,None,0,5.025481,...,-9.220297,0.008368,None,0.000000,0.001193,[],72.510497,NaN,setyawan_curtarolo=BandStructureSummaryData(ta...,95.525895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,OCCOCCOCCO,[edge=<Edge.K: 'K'> absorbing_element=Element ...,"[[1.0653492 3.74161971 9.54036525] C, [3.6986...",None,False,False,None,-2.5159,0,None,...,-8.069085,0.508996,None,0.000000,None,[],None,None,setyawan_curtarolo=BandStructureSummaryData(ta...,None
478,O,None,"[[3.85699327 0.45543037 4.22059147] O, [ 5.294...",None,True,False,None,-3.5005,0,None,...,-4.031695,0.914994,None,0.000000,None,[],None,None,None,None
479,OCC(O)CO,None,"[[2.28552632 3.12606474 4.78784431] C, [0.1999...",None,False,False,None,-5.3131,0,None,...,-7.746193,0.669242,None,0.000244,None,[],None,None,setyawan_curtarolo=BandStructureSummaryData(ta...,None
480,CCCCCC,[edge=<Edge.K: 'K'> absorbing_element=Element ...,"[[0. 0. 1.95076825] C, [0. ...",None,False,False,0.019117,None,0,5.025481,...,-9.220297,0.008368,None,0.000000,0.001193,[],72.510497,None,setyawan_curtarolo=BandStructureSummaryData(ta...,95.525895


In [15]:
df_mat_proj.to_csv("df_mat_proj.csv", index=False)


*Let's move on to RDKit*

In [16]:
rdkit_descriptors = [descriptor[0] for descriptor in Chem.Descriptors._descList]


In [17]:
len(rdkit_descriptors)


208

We can get about 200 descriptors from RDKit, let's create a molecular descriptor calculator which is basically a set of functions, from which we can get descriptors

In [18]:
descriptor_calculator_rdkit = MoleculeDescriptors.MolecularDescriptorCalculator(
    rdkit_descriptors
)


In [19]:
mols = []
names = []
descriptors_from_rdkit = []


Firstly, we have to convert smiles to mol, as it is used by RDKit and then we can use it to calculate the descriptors


In [ ]:
for name in tqdm(df.normalised_name):
    try:
        mol = Chem.MolFromSmiles(name)
        if mol is not None:
            mols += [mol]
            names += [name]
            descriptors_from_rdkit += [descriptor_calculator_rdkit.CalcDescriptors(mol)]
    except:
        pass


We can store our descriptors in a dataframe

In [21]:
df_rdkit = pd.DataFrame(columns=["normalised_name"] + ["mol"] + rdkit_descriptors)


In [22]:
for i, descriptor in enumerate(descriptors_from_rdkit):
    df_rdkit.loc[len(df_rdkit.index)] = [names[i]] + [mols[i]] + list(descriptor)


In [23]:
# Dataframe with all descriptors from RDKit
df_rdkit


,normalised_name,mol,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC(C)c1ccc(C)cc1,<rdkit.Chem.rdchem.Mol object at 0x000001F7822...,2.212963,0.653333,2.212963,0.653333,0.553258,134.222000,120.110,134.109550,...,0,0,0,0,0,0,0,0,0,0
1,O([2H])[2H],<rdkit.Chem.rdchem.Mol object at 0x000001F7823...,5.375000,2.750000,5.375000,2.750000,0.328547,20.027204,15.999,20.023118,...,0,0,0,0,0,0,0,0,0,0
2,c1ccncc1,<rdkit.Chem.rdchem.Mol object at 0x000001F7823...,3.784722,1.750000,3.784722,1.750000,0.453148,79.102000,74.062,79.042199,...,0,0,0,0,0,0,0,0,0,0
3,N.N.[Cl-].[Cl-].[Pt++],<rdkit.Chem.rdchem.Mol object at 0x000001F7822...,0.000000,0.000000,0.000000,0.000000,0.468143,300.046000,293.998,298.955595,...,0,0,0,0,0,0,0,0,0,0
4,Cn1nnnc1SCC2=C(N3[C@H](SC2)[C@H](NC(=O)[C@H](O...,<rdkit.Chem.rdchem.Mol object at 0x000001F7822...,12.713521,-1.418433,12.713521,0.078255,0.371888,462.513000,444.369,462.078010,...,2,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,CC(O)=O,<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,9.000000,-0.833333,9.000000,0.833333,0.429883,60.052000,56.020,60.021129,...,0,0,0,0,0,0,0,0,0,0
3401,[N-]=[Zn].[N-]=[Zn].[Zn+2],<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,7.125000,0.000000,7.125000,0.000000,0.539931,224.184000,224.184,219.793575,...,0,0,0,0,0,0,0,0,0,0
3402,OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C...,<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,9.997826,-2.221748,9.997826,0.668633,0.237891,342.297000,320.121,342.116212,...,0,0,0,0,0,0,0,0,0,0
3403,COC(=O)[C](C)C,<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,10.252315,-0.236111,10.252315,0.236111,0.455942,101.125000,92.053,101.060255,...,0,0,0,0,0,0,0,0,0,0


*Descriptors from Mordred*

Downloading descriptors from mordred is very easy, we can even use our previous dataframe and it will still look pretty


In [24]:
descriptor_calculator_mordred = Calculator(descriptors)

In [25]:
len(descriptor_calculator_mordred.descriptors)

1826

There are lots of descriptors in mordred


In [ ]:
df_mordred = descriptor_calculator_mordred.pandas(df_rdkit.mol)

Now we can merge these descriptors with RDKit dataframe

In [27]:
df_united = pd.concat([df_rdkit, df_mordred], axis=1)


Now we have a united dataframe with mordred and RDKit descriptors

In [28]:
df_united

,normalised_name,mol,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,CC(C)c1ccc(C)cc1,<rdkit.Chem.rdchem.Mol object at 0x000001F7822...,2.212963,0.653333,2.212963,0.653333,0.553258,134.222000,120.110,134.109550,...,8.815964,39.140584,134.109550,5.587898,120,11,46.0,50.0,4.333333,2.277778
1,O([2H])[2H],<rdkit.Chem.rdchem.Mol object at 0x000001F7823...,5.375000,2.750000,5.375000,2.750000,0.328547,20.027204,15.999,20.023118,...,4.174387,17.310771,20.023118,6.674373,4,0,6.0,4.0,2.25,1.000000
2,c1ccncc1,<rdkit.Chem.rdchem.Mol object at 0x000001F7823...,3.784722,1.750000,3.784722,1.750000,0.453148,79.102000,74.062,79.042199,...,7.627057,30.941317,79.042199,7.185654,27,3,24.0,24.0,1.5,1.500000
3,N.N.[Cl-].[Cl-].[Pt++],<rdkit.Chem.rdchem.Mol object at 0x000001F7822...,0.000000,0.000000,0.000000,0.000000,0.468143,300.046000,293.998,298.955595,...,0.000000,5.000000,298.955595,27.177781,1000000000,0,0.0,0.0,divide by zero encountered in power (mZagreb1),0.000000
4,Cn1nnnc1SCC2=C(N3[C@H](SC2)[C@H](NC(=O)[C@H](O...,<rdkit.Chem.rdchem.Mol object at 0x000001F7822...,12.713521,-1.418433,12.713521,0.078255,0.371888,462.513000,444.369,462.078010,...,10.612237,81.043479,462.078010,9.430163,2914,49,166.0,200.0,10.583333,6.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,CC(O)=O,<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,9.000000,-0.833333,9.000000,0.833333,0.429883,60.052000,56.020,60.021129,...,6.188264,24.179697,60.021129,7.502641,9,0,12.0,9.0,3.111111,1.000000
3401,[N-]=[Zn].[N-]=[Zn].[Zn+2],<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,7.125000,0.000000,7.125000,0.000000,0.539931,224.184000,224.184,219.793575,...,1.609438,13.047190,219.793575,43.958715,800000002,0,4.0,2.0,divide by zero encountered in power (mZagreb1),2.000000
3402,OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C...,<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,9.997826,-2.221748,9.997826,0.668633,0.237891,342.297000,320.121,342.116212,...,10.247042,72.317821,342.116212,7.602582,1110,43,120.0,147.0,10.451389,5.291667
3403,COC(=O)[C](C)C,<rdkit.Chem.rdchem.Mol object at 0x000001F7891...,10.252315,-0.236111,10.252315,0.236111,0.455942,101.125000,92.053,101.060255,...,7.890957,32.688753,101.060255,6.316266,46,6,26.0,26.0,4.472222,1.777778


In [29]:
df_united.to_csv('df_rdkit.csv', index=False)

**Data preprocessing**

We can concatenate all of our dataframes now

In [ ]:
df = pd.read_excel('data_mod1.xlsx')
df_united = pd.read_csv('df_rdkit.csv')
df_mat_proj = pd.read_csv('df_mat_proj.csv')

In [30]:
# Lets drop duplicates before merging
df = df.drop_duplicates(['normalised_name'])
df_united = df_united.drop_duplicates(['normalised_name'])
df_mat_proj= df_mat_proj.drop_duplicates(['normalised_name'])

In [32]:
# We can merge all 3 datasets in one
final_df = pd.concat(
    [
        df.set_index("normalised_name"),
        df_mat_proj.set_index("normalised_name"),
        df_united.set_index("normalised_name"),
    ],
    axis=1,
).reset_index()


In [34]:
final_df.to_csv('final_df.csv', index=False)

Let's check initial shape of our dataframe and the number of None values in descriptors before preprocessing

In [35]:
final_df.shape

(1520, 2096)

In [37]:
final_df.iloc[:,11:].isna().sum().sum()

1346013

In [38]:
# Split dataset in two parts to work with descriptors only
df_desc = final_df.iloc[:, 11:]
df_info = final_df.iloc[:, :11]


We have to convert all objects in descriptors to numeric values, as it's mostly errors. Same thing with infinite numbers

In [39]:
df_desc = df_desc.apply(pd.to_numeric, errors='coerce')
df_desc = df_desc.replace([np.inf, -np.inf], np.nan)

Let's check the number of None values

In [40]:
df_desc.isna().sum().sum()

1859486

Much more None values, now let's make final df back and remove rows with >50% None values

In [41]:
final_df = pd.concat([df_info, df_desc], axis=1)

In [42]:
final_df = final_df.dropna(axis = 0, thresh=int(0.5*final_df.shape[1])).reset_index(drop=True)

In [43]:
final_df.shape

(818, 2096)

In [45]:
final_df.iloc[:,11:].isna().sum().sum()

488739

We got rid of some Nones, but now our dataset has only 818 rows. Let's split it one more time to remove mostly empty columns

In [47]:
df_desc = final_df.iloc[:,11:]
df_info = final_df.iloc[:,:11]

We can also drop columns in which there are only one value and standard deviation equals zero

In [48]:
df_desc = df_desc.drop(df_desc.std()[(df_desc.std() == 0)].index, axis=1)
df_desc = df_desc.dropna(axis = 1, thresh=int(0.75*df_desc.shape[0]))

In [49]:
df_desc.shape

(818, 1306)

We still have more than 1000 descriptors

In [50]:
final_df = pd.concat([df_info, df_desc], axis=1)

It's time to fill in the missing values and apply scaling

In [51]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [52]:
df_desc = final_df.iloc[:,11:]
df_info = final_df.iloc[:,:11]

In [53]:
df_desc.isna().sum().sum()

51916

In [54]:
knn = KNNImputer()

In [55]:
df_filled = knn.fit_transform(df_desc)

In [56]:
df_desc = pd.DataFrame(df_filled, columns=df_desc.columns)

In [57]:
df_desc.isna().sum().sum()

0

Now we don't have None values, let's apply scaling

In [58]:
scaler = MinMaxScaler()

In [59]:
df_descritors_scaled = scaler.fit_transform(df_desc)

In [60]:
df_desc = pd.DataFrame(df_descritors_scaled, columns=df_desc.columns)

In [62]:
final_df = pd.concat([df_info, df_desc], axis=1)

In [63]:
final_df.shape

(818, 1317)

In [ ]:
final_df.to_csv('hw1_dataset.csv', index=False)